In [2]:
%pip install psycopg2-binary
%pip install pandas
%pip install numpy
%pip install scipy
%pip install ipywidgets
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


----
# Metadata Wide Association Study (MWAS)


In [3]:
# import required libraries
import psycopg2
import pandas as pd
import numpy as np
import scipy.stats as stats
import ipywidgets as widgets
import requests
import warnings
import io
import csv

warnings.filterwarnings('ignore') # FIX THIS LATERE TO NOT IGNORE WARNINGS

# database connection details
host = "serratus-aurora-20210406.cluster-ro-ccz9y6yshbls.us-east-1.rds.amazonaws.com"
database = "summary"
user = "public_reader"
password = "serratus"

# ensure connection is valid then close
try:
    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    conn.close()
    print("Successfully connected to Serratus 🗻")
except:
    print("Error conecting to database.")

Successfully connected to Serratus 🗻


- for initial MWAS run, want to use the `n_reads` column from `rfamily` table
- analysis should be run on all bioprojects that appear in the `rfamily` table
    - mapping from runs to bioprojects found in the `srarun` table
- metadata csvs for bioprojects are pre generated and stored on s3
- analysis should look for associations between metadata columns and `n_reads` columns for all viruses for all bioprojects

In [4]:
'''
rfamily database contains the number of reads of the virus for each run and the corresponding family_group

srarun database contains the number of spots for the run and the corresponding bio_sample, sample, bio_project, scientific_name

'''

rfam_q = ''' SELECT run_id, family_group, percent_identity, n_reads
                FROM rfamily
            '''

srarun_q = ''' SELECT run, bio_sample, sample, bio_project, scientific_name, spots
                FROM srarun
            '''

with psycopg2.connect(host=host, database=database, user=user, password=password) as conn:
    rfam_df = pd.read_sql(rfam_q, conn)
    srarun_df = pd.read_sql(srarun_q, conn)

In [5]:
rfam_runs = set(rfam_df['run_id'].unique())
srarun_runs = set(srarun_df['run'].unique())

# print any runs that are in the rfam database but not in the srarun database
diff = rfam_runs - srarun_runs
print("Runs in rfam but not srarun: ", len(diff))
print('Number of coresponding rows in rfam: ', len(rfam_df[rfam_df['run_id'].isin(diff)]))
print(diff)
# remove the runs that are in the rfam database but not in the srarun database as we cannot get associated metadata
rfam_df = rfam_df[~rfam_df['run_id'].isin(diff)]

# store the difference to be run in the python script
diff = {'SRR10566129', 'SRR12149727', 'SRR10566134', 'SRR10505610', 'SRR10566126', 'SRR12149724', 'SRR10505607', 'SRR10505604', 'SRR10566125', 'SRR10566133', 'SRR10566128', 'SRR10566123', 'SRR10566130', 'SRR10505615', 'SRR10566121', 'SRR10505601', 'SRR10566122', 'SRR10566132', 'SRR10566127', 'SRR10505611', 'SRR12149728', 'SRR10566131', 'SRR10566124'}

Runs in rfam but not srarun:  23
Number of coresponding rows in rfam:  63
{'SRR10566127', 'SRR10505607', 'SRR10566128', 'SRR10566124', 'SRR10566125', 'SRR10566134', 'SRR10566131', 'SRR12149724', 'SRR10505601', 'SRR10566122', 'SRR12149728', 'SRR10505611', 'SRR10505604', 'SRR10566130', 'SRR10566121', 'SRR12149727', 'SRR10505615', 'SRR10566132', 'SRR10566123', 'SRR10566126', 'SRR10566129', 'SRR10505610', 'SRR10566133'}


In [6]:
# print the number of unique values in family_group
# want to run the analysis on all family groups
print("Number of unique family groups: ", len(rfam_df['family_group'].unique()))
family_groups = list(rfam_df['family_group'].unique())

Number of unique family groups:  3282


In [7]:
# split the family_groups into groups of 200 and write to files in mwas_rfam folder
# allow the analysis to be run in parallel
count = 0
file_num = 0
while count < len(family_groups):
    with open('mwas_rfam/family_groups/family_groups_{}.txt'.format(file_num), 'w') as f:
        for i in range(count, count + 200):
            if i < len(family_groups):
                f.write(family_groups[i] + '\n')
            else:
                break
    count += 200
    file_num += 1

In [8]:
'''
MWAS file should take in a file with a list of family groups on a new line
'''

import sys

# read in the file name from the command line argument
# if len(sys.argv) != 2:
#     print("Please enter a file name.")
#     sys.exit(1)
# family_groups_file = sys.argv[1]

# files are located in the family_groups directory
# with open('family_groups/' + family_file, 'r') as f:
#     for family in f:
#         print(family)
#         break

family = 'Totiviridae-10'
family = family.strip()

# get the number of reads for the family group
family_df = rfam_df[rfam_df['family_group'] == family]

In [9]:
# merge the family_df with the srarun_df to get the number of spots for each run
mwas_df = pd.merge(family_df, srarun_df, left_on='run_id', right_on='run')

# check that the merge was successful (no runs lost)
assert len(mwas_df) == len(family_df) 

# count number of runs with 0 spots
num_zero_spots = len(mwas_df[mwas_df['spots'] == 0])
print("Number of runs with 0 spots: ", num_zero_spots) # LOG THIS
# replace 0 with 1,000,000 spots if necessary (ARBITRARY CHOICE)
if num_zero_spots > 0:
    # mwas_df['spots'] = mwas_df['spots'].replace(0, 1000000)
    mwas_df.loc[mwas_df['spots'] == 0, 'spots'] = 1000000
    
mwas_df

Number of runs with 0 spots:  0


,run_id,family_group,percent_identity,n_reads,run,bio_sample,sample,bio_project,scientific_name,spots
0,SRR12369764,Totiviridae-10,71,6,SRR12369764,SAMN15702630,SRS7129764,PRJNA650304,Mus musculus,95455823
1,SRR12369805,Totiviridae-10,95,4,SRR12369805,SAMN15702699,SRS7129781,PRJNA650306,Homo sapiens,50136340
2,SRR12362004,Totiviridae-10,73,19,SRR12362004,SAMN15689828,SRS7123283,PRJNA649960,food metagenome,43805632
3,SRR12214566,Totiviridae-10,50,1,SRR12214566,SAMN15524028,SRS7000050,PRJNA646066,Homo sapiens,62188730
4,SRR12333764,Totiviridae-10,100,2,SRR12333764,SAMN15654926,SRS7097823,PRJNA648996,Rattus norvegicus,43671791
...,...,...,...,...,...,...,...,...,...,...
76448,SRR9995119,Totiviridae-10,65,56,SRR9995119,SAMN12598752,SRS5285451,PRJNA560721,Apis cerana cerana,152058019
76449,SRR9995126,Totiviridae-10,82,147,SRR9995126,SAMN12496165,SRS5285461,PRJNA558612,Plasmopara viticola,38931600
76450,SRR9995127,Totiviridae-10,85,59,SRR9995127,SAMN12496164,SRS5285460,PRJNA558612,Plasmopara viticola,37833782
76451,SRR9995128,Totiviridae-10,87,94,SRR9995128,SAMN12496163,SRS5285459,PRJNA558612,Plasmopara viticola,42709834


In [10]:
# get the unique bio_projects
bio_projects = mwas_df['bio_project'].unique()
print("Number of unique bio_projects: ", len(bio_projects))

# # get the unique bio_samples
# bio_samples = mwas_df['bio_sample'].unique()
# print("Number of unique bio_samples: ", len(bio_samples))


Number of unique bio_projects:  16957


In [11]:
# set up variables for API call
base_url = 'https://serratus-biosamples.s3.us-east-1.amazonaws.com/bioprojects_csv/'
headers = {'Host': 'serratus-biosamples.s3.amazonaws.com'}

In [12]:
srarun_samples = set(srarun_df['bio_sample'].unique())

In [30]:
# set up columns for mwas output
mwas_output_cols = ['bioproject_id', 'metadata_field', 'metadata_term', 'n_true', 'n_false', 'mean_rpm_true', 'mean_rpm_false', 'sd_rpm_true', 'sd_rpm_false', 'fold_change', 'p_value', 'z_score']
mwas_output_dict = {col: [] for col in mwas_output_cols}

# get all bioprojects corresponding to the runs in the family group
bioprojects = set(mwas_df['bio_project'].unique())
# remove None from the bioprojects if it exists
if None in bioprojects:
    bioprojects.remove(None)

# bioprojects = {'PRJNA398546'}

error_count = 0
# iterate through each bioproject
for bioproject_id in bioprojects:
    url = base_url + bioproject_id + '.csv'
    # make API call
    response = requests.get(url, headers=headers)
    # check if the API call was successful
    if response.status_code != 200:
        print(f'Error: {response.status_code} for {bioproject_id}') # LOG THIS
        error_count += 1
        continue
    result_string = response.content.decode('utf-8')
    result_string = io.StringIO(result_string)
    # read in the csv file
    bioproject_df = pd.read_csv(result_string, quoting=csv.QUOTE_NONE)

    # skip if there is a single biosample row
    if len(bioproject_df) == 1:
        continue

    # remove all columns starting with GEO Sample * FIND A BETTER WAY TO DO THIS LATER
    bioproject_df = bioproject_df.loc[:, ~bioproject_df.columns.str.startswith('GEO Sample')]
    
    display(bioproject_df)
    break
    # add the SRR accessions and read counts to the bioproject_df
    try:
        # TAKES VERY LONG
        # bioproject_df = bioproject_df.merge(mwas_df, left_on='biosample_id', right_on='bio_sample', how='left')
        bioproject_df = bioproject_df.join(mwas_df.set_index('bio_sample'), on='biosample_id', how='left')
    except Exception as e:
        print(f'Error: {e} for {bioproject_id}')

    # calculate the RPM for each run
    bioproject_df['rpm'] = bioproject_df['n_reads'] / bioproject_df['spots'] * 1000000
 

    bioproject_df = bioproject_df.merge(srarun_df, left_on='biosample_id', right_on='bio_sample', how='left')
    bioproject_df.drop('bio_sample', inplace=True, axis=1)

    for col in {'run', 'sample', 'bio_project', 'scientific_name', 'spots'}:
        bioproject_df[col + '_x'] = bioproject_df[col + '_y'].combine_first(bioproject_df[col + '_x'])
        bioproject_df = bioproject_df.drop(col + '_y', axis=1)
    
    bioproject_df.loc[~bioproject_df['run_x'].isna() & bioproject_df['rpm'].isna(), 'rpm'] = 0

    # display(bioproject_df)
    # get the target columns
    remove_cols = {'biosample_id', 'publication_date', 'last_update', 'n_reads', 'spots', 'percent_identity', 'rpm'} # TO CHANGE AS NEEDED
    target_cols = set(bioproject_df.columns) - remove_cols

    metadata_counts = {}
    binary_cols = set()

    num_rows = bioproject_df.shape[0]

    # iterate through each column and store corresponding value counts
    for target_col in target_cols:
        value_counts = bioproject_df[target_col].value_counts()
        if len(value_counts) <= 1 or len(value_counts) == num_rows:
            # skip if there is only 1 value or if there is a value for each row
            continue
        else:
            # at least one value is repeated
            if len(value_counts) == 2:
                binary_cols.add(target_col)
            metadata_counts[target_col] = value_counts.to_dict()

    print(metadata_counts)
    print(binary_cols)

    existing_runs = []
    target_col_runs = {}
    for target_col, value_counts in metadata_counts.items():
        for target_term, count in value_counts.items():
            if count == 1:
                # skip if there is only 1 instance of the term
                continue
            else:
                target_run_ids = set(bioproject_df[bioproject_df[target_col] == target_term]['run_x'])
                if target_run_ids in existing_runs:
                    # target_run_ids already in the set so aggregate the column name with the existing column
                    target_col_name = target_col + '\t' + str(target_term)
                    # find the existing key that has the same value in the dict
                    existing_key = list(target_col_runs.keys())[list(target_col_runs.values()).index(target_run_ids)]
                    target_col_name = target_col_name + '\r' + existing_key
                    # update the existing key to the new key by separating with '\r'
                    target_col_runs[target_col_name] = target_col_runs.pop(existing_key)
                else:
                    # target_run_ids is not in the set so add to the set and dict
                    existing_runs.append(target_run_ids)
                    # ASSUMES THAT '\t' IS NOT IN THE TERM
                    target_col_runs[target_col + '\t' + str(target_term)] = target_run_ids 

    break


Error: 404 for PRJNA780516
Error: 404 for PRJNA652526
Error: 404 for PRJNA749564
Error: 404 for PRJNA843055
Error: 404 for PRJEB40724
Error: 404 for PRJEB35724
Error: 404 for PRJNA489360
Error: 404 for PRJNA438476
Error: 404 for PRJNA695250
Error: 404 for PRJNA493215
Error: 404 for PRJNA523289
Error: 404 for PRJNA429669
Error: 404 for PRJNA448899
Error: 404 for PRJNA635918
Error: 404 for PRJNA600020
Error: 404 for PRJNA311568
Error: 404 for PRJNA713391
Error: 404 for PRJNA317331
Error: 404 for PRJNA307566
Error: 404 for PRJNA667658
Error: 404 for PRJNA482439
Error: 404 for PRJNA510940
Error: 404 for PRJEB21166
Error: 404 for PRJNA666649
Error: 404 for PRJEB31605
Error: 404 for PRJNA314902
Error: 404 for PRJEB38391
Error: 404 for PRJNA753113
Error: 404 for PRJNA510747
Error: 404 for PRJNA206070
Error: 404 for PRJNA564313
Error: 404 for PRJNA645125
Error: 404 for PRJNA730902
Error: 404 for PRJEB39649
Error: 404 for PRJNA858444
Error: 404 for PRJNA895362
Error: 404 for PRJNA475756
Error: 

,biosample_id,submission_date,last_update,publication_date,access,BioSample_db,Sample name_db,title,organism_taxonomy_id,organism_taxonomy_name,...,sample_type,metagenome-source,derived_from,bioproject,status,when,paragraph,note,strain,host
0,SAMN18738615,2021-04-13T23:35:20.873,2022-05-02T00:50:26.223,2022-05-02T00:50:26.223,public,SAMN18738615,GYT0_bin.3,MIMAG Metagenome-assembled Genome sample from ...,1871037,Flavobacteriaceae bacterium,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
1,SAMN18738616,2021-04-13T23:35:20.883,2022-05-02T00:50:26.223,2022-05-02T00:50:26.223,public,SAMN18738616,GYT0_bin.4,MIMAG Metagenome-assembled Genome sample from ...,1899355,Oceanospirillaceae bacterium,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
2,SAMN18738617,2021-04-13T23:35:20.897,2022-05-02T00:50:26.237,2022-05-02T00:50:26.237,public,SAMN18738617,GYT0_bin.8,MIMAG Metagenome-assembled Genome sample from ...,1969726,Sulfurovum sp.,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
3,SAMN18738628,2021-04-13T23:35:21.027,2022-05-02T00:50:26.347,2022-05-02T00:50:26.347,public,SAMN18738628,GYT0_bin.46,MIMAG Metagenome-assembled Genome sample from ...,2021391,Flavobacteriales bacterium,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
4,SAMN18738623,2021-04-13T23:35:20.967,2022-05-02T00:50:26.300,2022-05-02T00:50:26.300,public,SAMN18738623,GYT0_bin.28,MIMAG Metagenome-assembled Genome sample from ...,1284658,Planktomarina temperata,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
5,SAMN18738629,2021-04-13T23:35:21.040,2022-05-02T00:50:26.347,2022-05-02T00:50:26.347,public,SAMN18738629,GYT0_bin.49,MIMAG Metagenome-assembled Genome sample from ...,1872504,Amylibacter sp.,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
6,SAMN18738621,2021-04-13T23:35:20.943,2022-05-02T00:50:26.270,2022-05-02T00:50:26.270,public,SAMN18738621,GYT0_bin.20,MIMAG Metagenome-assembled Genome sample from ...,576131,Lentibacter algarum,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
7,SAMN18738633,2021-04-13T23:35:21.087,2022-05-02T00:50:26.393,2022-05-02T00:50:26.393,public,SAMN18738633,GY13CH_bin.8,MIMAG Metagenome-assembled Genome sample from ...,2021391,Flavobacteriales bacterium,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073771,SAMN17073772,"and SAMN17073773""",13C labeled microbioorganism,SAMN17073770,NaN,NaN
8,SAMN18738614,2021-04-13T23:35:20.863,2022-05-02T00:50:26.207,2022-05-02T00:50:26.207,public,SAMN18738614,GYT0_bin.2,MIMAG Metagenome-assembled Genome sample from ...,2212373,Yoonia sp.,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN
9,SAMN18738638,2021-04-13T23:35:21.220,2022-05-02T00:50:26.440,2022-05-02T00:50:26.440,public,SAMN18738638,GY13CL_bin.9,MIMAG Metagenome-assembled Genome sample from ...,2021391,Flavobacteriales bacterium,...,metagenomic assembly,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073771,SAMN17073772,"and SAMN17073773""",unlabeled microbioorganism,SAMN17073770,NaN,NaN


In [40]:
# bioproject_df[bioproject_df['biosample_id'].isin(srarun_samples)]
bioproject_df.columns
# bioproject_df.head()
mwas_df[mwas_df['bio_project'] == 'PRJNA685000']
mwas_df
'PRJNA558612'

'PRJNA558612'

In [41]:
srarun_df

,run,bio_sample,sample,bio_project,scientific_name,spots
0,SRR11167643,SAMN14170104,SRS6205642,PRJNA608425,Homo sapiens,1555981
1,SRR11167644,SAMN14170103,SRS6205675,PRJNA608425,Homo sapiens,2103053
2,SRR11167645,SAMN14170102,SRS6205677,PRJNA608425,Homo sapiens,1632613
3,SRR11167646,SAMN14170101,SRS6205676,PRJNA608425,Homo sapiens,1719397
4,SRR11167647,SAMN14170100,SRS6205678,PRJNA608425,Homo sapiens,1169500
...,...,...,...,...,...,...
7680421,SRR11167638,SAMN14170068,SRS6205637,PRJNA608425,Homo sapiens,1631235
7680422,SRR11167639,SAMN14170067,SRS6205638,PRJNA608425,Homo sapiens,2173094
7680423,SRR11167640,SAMN14170107,SRS6205639,PRJNA608425,Homo sapiens,1821417
7680424,SRR11167641,SAMN14170106,SRS6205640,PRJNA608425,Homo sapiens,2190953


In [100]:
family_df
# merge the n_reads and run_id columns from family df with the bio_sample from srarun_df
df = pd.merge(family_df[['run_id', 'n_reads']], srarun_df[['run', 'bio_sample', 'spots']], left_on='run_id', right_on='run')
df

,run_id,n_reads,run,bio_sample,spots
0,SRR12369764,6,SRR12369764,SAMN15702630,95455823
1,SRR12369805,4,SRR12369805,SAMN15702699,50136340
2,SRR12362004,19,SRR12362004,SAMN15689828,43805632
3,SRR12214566,1,SRR12214566,SAMN15524028,62188730
4,SRR12333764,2,SRR12333764,SAMN15654926,43671791
...,...,...,...,...,...
76448,SRR9995119,56,SRR9995119,SAMN12598752,152058019
76449,SRR9995126,147,SRR9995126,SAMN12496165,38931600
76450,SRR9995127,59,SRR9995127,SAMN12496164,37833782
76451,SRR9995128,94,SRR9995128,SAMN12496163,42709834


In [78]:
srarun_biosamples = {}
# create a mapping from serratus annotated biosamples to 0 (default n_reads)
def create_srarun_biosamples(row):
    srarun_biosamples[row['bio_sample']] = 0

srarun_df.apply(create_srarun_biosamples, axis=1)
len(srarun_biosamples)

5783686

In [105]:
# use df to create a dictionary mapping bio_sample to n_reads, if there are multiple runs for a bio_sample, set the n_reads to a list of the n_reads for each run
biosample_count_dict = {}
biosample_runs_dict = {}
biosample_spots_dict = {}
def add_to_dict(row, biosample_count_dict=biosample_count_dict, biosample_runs_dict=biosample_runs_dict):
    if row['bio_sample'] in biosample_count_dict:
        biosample_count_dict[row['bio_sample']].append(row['n_reads'])
        biosample_runs_dict[row['bio_sample']].append(row['run_id'])
        biosample_spots_dict[row['bio_sample']].append(row['spots'])
    else:
        biosample_count_dict[row['bio_sample']] = [row['n_reads']]
        biosample_runs_dict[row['bio_sample']] = [row['run_id']]
        biosample_spots_dict[row['bio_sample']] = [row['spots']]

df.apply(add_to_dict, axis=1)
# set the n_reads to the average of the n_reads for each run
for biosample, n_reads in biosample_count_dict.items():
    biosample_count_dict[biosample] = np.mean(n_reads)

# set the spots to the average of the spots for each run
for biosample, spots in biosample_spots_dict.items():
    biosample_spots_dict[biosample] = np.mean(spots)

bio_sample_dict.__len__()

67605

In [104]:
biosample_spots_dict

{'SAMN15702630': [95455823],
 'SAMN15702699': [50136340],
 'SAMN15689828': [43805632],
 'SAMN15524028': [62188730],
 'SAMN15654926': [43671791],
 'SAMN15524027': [43043341],
 'SAMN14512602': [82481666],
 'SAMN14513312': [68640917],
 'SAMD00013963': [92855805, 55852888, 75320405, 76946227, 104954914, 26867713],
 'SAMD00008650': [21067867],
 'SAMD00015768': [59435004, 70715306],
 'SAMD00013613': [103359417],
 'SAMD00013610': [119787306, 74869816],
 'SAMD00013607': [89923286, 54326141, 78915761],
 'SAMD00013606': [101767309, 57654250, 58741804],
 'SAMD00013612': [75008973, 113132803],
 'SAMD00000565': [15971589],
 'SAMD00013688': [72857508],
 'SAMD00013675': [67647972],
 'SAMD00013690': [79927287],
 'SAMD00013691': [72635803],
 'SAMD00013684': [55743037],
 'SAMD00013677': [35731928],
 'SAMD00013674': [60200625],
 'SAMD00013682': [56661120],
 'SAMD00013681': [49585483],
 'SAMD00013680': [58137622],
 'SAMD00013693': [191708116],
 'SAMD00003367': [82191816],
 'SAMD00003362': [33752426],
 'SA

In [86]:
srarun_df[srarun_df['bio_project'] == 'PRJNA685000']

,run,bio_sample,sample,bio_project,scientific_name,spots
1016606,SRR13262723,SAMN17073773,SRS7890119,PRJNA685000,seawater metagenome,46900054
1016607,SRR13262724,SAMN17073772,SRS7890118,PRJNA685000,seawater metagenome,47002169
1016608,SRR13262725,SAMN17073771,SRS7890117,PRJNA685000,seawater metagenome,43934558
1016609,SRR13262726,SAMN17073770,SRS7890116,PRJNA685000,seawater metagenome,46829989
1016610,SRR13262727,SAMN17073769,SRS7890115,PRJNA685000,seawater metagenome,46620445


In [95]:
srarun_biosamples_new = srarun_biosamples.copy()
srarun_biosamples_new['SAMN18738615'] = 0
srarun_biosamples_new['SAMN18738616'] = 0
srarun_biosamples_new['SAMN18738617'] = 0
biosample_count_dict_new = biosample_count_dict.copy()
biosample_count_dict_new['SAMN18738615'] = 1000
biosample_count_dict_new['SAMN18738628'] = 1004560


# bioproject_df['n_reads'] = bioproject_df['biosample_id'].map(srarun_biosamples_new)
# bioproject_df['n_reads'] = bioproject_df['biosample_id'].map(biosample_count_dict)
# bioproject_df['biosample_id']

def help_n_reads(row):
    # try to get the n_reads from biosample_count_dict, otherwise try from srarun_biosamples_new, otherwise return np.nan
    try:
        return biosample_count_dict_new[row['biosample_id']]
    except:
        try:
            return srarun_biosamples_new[row['biosample_id']]
        except:
            return np.nan

bioproject_df['n_reads'] = bioproject_df.apply(help_n_reads, axis=1)
bioproject_df

,biosample_id,submission_date,last_update,publication_date,access,BioSample_db,Sample name_db,title,organism_taxonomy_id,organism_taxonomy_name,...,metagenome-source,derived_from,bioproject,status,when,paragraph,note,strain,host,n_reads
0,SAMN18738615,2021-04-13T23:35:20.873,2022-05-02T00:50:26.223,2022-05-02T00:50:26.223,public,SAMN18738615,GYT0_bin.3,MIMAG Metagenome-assembled Genome sample from ...,1871037,Flavobacteriaceae bacterium,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,1000.0
1,SAMN18738616,2021-04-13T23:35:20.883,2022-05-02T00:50:26.223,2022-05-02T00:50:26.223,public,SAMN18738616,GYT0_bin.4,MIMAG Metagenome-assembled Genome sample from ...,1899355,Oceanospirillaceae bacterium,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,0.0
2,SAMN18738617,2021-04-13T23:35:20.897,2022-05-02T00:50:26.237,2022-05-02T00:50:26.237,public,SAMN18738617,GYT0_bin.8,MIMAG Metagenome-assembled Genome sample from ...,1969726,Sulfurovum sp.,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,0.0
3,SAMN18738628,2021-04-13T23:35:21.027,2022-05-02T00:50:26.347,2022-05-02T00:50:26.347,public,SAMN18738628,GYT0_bin.46,MIMAG Metagenome-assembled Genome sample from ...,2021391,Flavobacteriales bacterium,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,1004560.0
4,SAMN18738623,2021-04-13T23:35:20.967,2022-05-02T00:50:26.300,2022-05-02T00:50:26.300,public,SAMN18738623,GYT0_bin.28,MIMAG Metagenome-assembled Genome sample from ...,1284658,Planktomarina temperata,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,NaN
5,SAMN18738629,2021-04-13T23:35:21.040,2022-05-02T00:50:26.347,2022-05-02T00:50:26.347,public,SAMN18738629,GYT0_bin.49,MIMAG Metagenome-assembled Genome sample from ...,1872504,Amylibacter sp.,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,NaN
6,SAMN18738621,2021-04-13T23:35:20.943,2022-05-02T00:50:26.270,2022-05-02T00:50:26.270,public,SAMN18738621,GYT0_bin.20,MIMAG Metagenome-assembled Genome sample from ...,576131,Lentibacter algarum,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,NaN
7,SAMN18738633,2021-04-13T23:35:21.087,2022-05-02T00:50:26.393,2022-05-02T00:50:26.393,public,SAMN18738633,GY13CH_bin.8,MIMAG Metagenome-assembled Genome sample from ...,2021391,Flavobacteriales bacterium,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073771,SAMN17073772,"and SAMN17073773""",13C labeled microbioorganism,SAMN17073770,NaN,NaN,NaN
8,SAMN18738614,2021-04-13T23:35:20.863,2022-05-02T00:50:26.207,2022-05-02T00:50:26.207,public,SAMN18738614,GYT0_bin.2,MIMAG Metagenome-assembled Genome sample from ...,2212373,Yoonia sp.,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073770,SAMN17073771,SAMN17073772,"and SAMN17073773""",NaN,NaN,NaN,NaN
9,SAMN18738638,2021-04-13T23:35:21.220,2022-05-02T00:50:26.440,2022-05-02T00:50:26.440,public,SAMN18738638,GY13CL_bin.9,MIMAG Metagenome-assembled Genome sample from ...,2021391,Flavobacteriales bacterium,...,seawater metagenome,"""This BioSample is a metagenomic assembly obta...",SAMN17073771,SAMN17073772,"and SAMN17073773""",unlabeled microbioorganism,SAMN17073770,NaN,NaN,NaN
